In [1]:
#Mount Google Drive for input videos
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Install these libraries incase you don't have them installed 
!pip install sk-video
!pip install scipy==1.1.0
!pip install gdown
!pip install --upgrade gdown

     |████████████████████████████████| 2.3 MB 5.3 MB/s 
     |████████████████████████████████| 31.2 MB 172 kB/s 
  Attempting uninstall: scipy
    Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pymc3 3.11.4 requires scipy>=1.2.0, but you have scipy 1.1.0 which is incompatible.
plotnine 0.6.0 requires scipy>=1.2.0, but you have scipy 1.1.0 which is incompatible.
jax 0.3.4 requires scipy>=1.2.1, but you have scipy 1.1.0 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for gdown: filename=gdown-4.4.0-py3-none-any.whl size=14774 sha256=

In [3]:
#dataset link
#https://drive.google.com/file/d/1wxJf-xUjVcFtSgayGDU959rNG3Bj96LY/view?usp=sharing

In [4]:
%cd /content/drive/MyDrive

/content/drive/MyDrive


In [5]:
!gdown 1wxJf-xUjVcFtSgayGDU959rNG3Bj96LY

Downloading...
From: https://drive.google.com/uc?id=1wxJf-xUjVcFtSgayGDU959rNG3Bj96LY
To: /content/drive/MyDrive/view_point_dataset.rar
100% 518M/518M [00:03<00:00, 137MB/s]


In [6]:
!apt-get install rar

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  rar
0 upgraded, 1 newly installed, 0 to remove and 39 not upgraded.
Need to get 295 kB of archives.
After this operation, 799 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/multiverse amd64 rar amd64 2:5.5.0-1 [295 kB]
Fetched 295 kB in 1s (422 kB/s)
Selecting previously unselected package rar.
(Reading database ... 156210 files and directories currently installed.)
Preparing to unpack .../rar_2%3a5.5.0-1_amd64.deb ...
Unpacking rar (2:5.5.0-1) ...
Setting up rar (2:5.5.0-1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...


In [7]:
!mkdir '/content/drive/MyDrive/vp_pro'
!mkdir '/content/drive/MyDrive/vp_pro/out'
!mkdir '/content/drive/MyDrive/vp_pro/ext'
!mkdir '/content/drive/MyDrive/vp_pro/input'

In [8]:
!unrar x  "/content/drive/MyDrive/view_point_dataset.rar" "/content/drive/MyDrive/vp_pro/"


UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from /content/drive/MyDrive/view_point_dataset.rar

Creating    /content/drive/MyDrive/vp_pro/view_point_dataset          OK
Creating    /content/drive/MyDrive/vp_pro/view_point_dataset/boarding  OK
Extracting  /content/drive/MyDrive/vp_pro/view_point_dataset/boarding/777Yn0k-fLk.mp4       0%  1%  2%  OK 
Extracting  /content/drive/MyDrive/vp_pro/view_point_dataset/boarding/lPtFEa6idnA.mp4       3%  OK 
Extracting  /content/drive/MyDrive/vp_pro/view_point_dataset/boarding/ONtIoZxQQIk.mp4       4%  5%  6%  7%  8%  OK 
Extracting  /content/drive/MyDrive/vp_pro/view_point_dataset/boarding/pAPMCTDMotM.mp4       9% 10% 11% 12%  OK 
Creating    /content/drive/MyDrive/vp_pro/view_point_dataset/dog      OK
Extracting  /content/drive/MyDrive/vp_pro/view_point_dataset/dog/61clg15Wc2s.mp4      13% 14%  OK 
Extracting  /content/drive/M

In [9]:
#Importing Required Libraries
import numpy as np
import skvideo.io
import cv2
import os

In [10]:
#number of videos given to the model are 5
nov=3

In [11]:
#list of paths of the videos
def getListOfFiles(dirName):
    # create a list of file and sub directories 
    # names in the given directory 
    listOfFile = os.listdir(dirName)
    allFiles = list()
    # Iterate over all the entries
    for entry in listOfFile:
        # Create full path
        fullPath = os.path.join(dirName, entry)
        # If entry is a directory then get the list of files in this directory 
        if os.path.isdir(fullPath):
            allFiles = allFiles + getListOfFiles(fullPath)
        else:
            allFiles.append(fullPath)
                
    return allFiles

In [12]:
#path to videos
dirName = '/content/drive/MyDrive/vp_pro/view_point_dataset/Running';
# Get the list of all files in directory tree at given path
listOfFiles = getListOfFiles(dirName)

In [13]:
listOfFiles

['/content/drive/MyDrive/vp_pro/view_point_dataset/Running/6eCgHq_7GPU.mp4',
 '/content/drive/MyDrive/vp_pro/view_point_dataset/Running/jnv0O9riZqc.mp4',
 '/content/drive/MyDrive/vp_pro/view_point_dataset/Running/rClB45bU4YE.mp4']

In [14]:
#Decode predictions are made by the C3D model pretrainded on Sports1M
#Return decoded prediction along with confidence value

import numpy as np
from scipy.misc import imresize
from keras.utils.data_utils import get_file

C3D_MEAN_PATH = 'https://github.com/adamcasson/c3d/releases/download/v0.1/c3d_mean.npy'
#If you want to use general decoder uncomment this and comment the next line
#SPORTS1M_CLASSES_PATH = 'https://github.com/adamcasson/c3d/releases/download/v0.1/sports1M_classes.txt'
#Modidifed decoder for view point
SPORTS1M_CLASSES_PATH = 'https://github.com/sureshk-15/Video-Summarization-using-Convolutional-Neural-Network/raw/gh-pages/sports1M_classes.txt'


#Function is to Preprocess input Videos to required shape 
def preprocess_input(video):
   
    intervals = np.ceil(np.linspace(0, video.shape[0]-1, 16)).astype(int)
    frames = video[intervals]
    
    # Reshape to 128x171
    reshape_frames = np.zeros((frames.shape[0], 128, 171, frames.shape[3]))
    for i, img in enumerate(frames):
        img = imresize(img, (128,171), 'bicubic')
        reshape_frames[i,:,:,:] = img
        
    mean_path = get_file('c3d_mean.npy',
                         C3D_MEAN_PATH,
                         cache_subdir='models')
    
    # Subtract mean
    mean = np.load(mean_path)
    reshape_frames -= mean
    # Crop to 112x112
    reshape_frames = reshape_frames[:,8:120,30:142,:]
    # Add extra dimension for samples
    reshape_frames = np.expand_dims(reshape_frames, axis=0)
    
    return reshape_frames

#Function is  to decode prediction made by pretrained Model and return decoded prediction
def decode_predictions(preds):
    
    class_pred = []
    for x in range(preds.shape[0]):
        class_pred.append(np.argmax(preds[x]))
    
    labels_path = get_file('sports1M_classes.txt',
                           SPORTS1M_CLASSES_PATH,
                           cache_subdir='models')
    
    with open(labels_path, 'r') as f:
        labels = [lines.strip() for lines in f]
        
    decoded = [(labels[x],preds[i,x]) for i,x in enumerate(class_pred)]
    
    return decoded

In [15]:
# It's a 3D Convolutional Neural Network 
# Input to the model is 16 frames of video (16,112,112,3)
# Output of the model is that it Predicts index and Confidence score
# Weights- Pretrained on sports1M dataset

import skvideo.io
import keras.backend as K
from keras.models import Sequential
from keras.utils.data_utils import get_file
from keras.layers.core import Dense, Dropout, Flatten
#from sports1M_utils import preprocess_input, decode_predictions
from keras.layers.convolutional import Conv3D, MaxPooling3D, ZeroPadding3D

#Path for decoded weights
WEIGHTS_PATH = 'https://github.com/adamcasson/c3d/releases/download/v0.1/sports1M_weights_tf.h5'

def C3D(weights='sports1M'):
    """Instantiates a C3D Kerasl model
    
    Keyword arguments:
    weights -- weights to load into model. (default is sports1M)
    
    Returns:
    A Keras model.
    
    """
    
    if weights not in {'sports1M', None}:
        raise ValueError('weights should be either be sports1M or None')
    
    if K.image_data_format() == 'channels_last':
        shape = (16,112,112,3)
    else:
        shape = (3,16,112,112)
        
    model = Sequential()
    model.add(Conv3D(64, 3, activation='relu', padding='same', name='conv1', input_shape=shape))
    model.add(MaxPooling3D(pool_size=(1,2,2), strides=(1,2,2), padding='same', name='pool1'))
    
    model.add(Conv3D(128, 3, activation='relu', padding='same', name='conv2'))
    model.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2), padding='valid', name='pool2'))
    
    model.add(Conv3D(256, 3, activation='relu', padding='same', name='conv3a'))
    model.add(Conv3D(256, 3, activation='relu', padding='same', name='conv3b'))
    model.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2), padding='valid', name='pool3'))
    
    model.add(Conv3D(512, 3, activation='relu', padding='same', name='conv4a'))
    model.add(Conv3D(512, 3, activation='relu', padding='same', name='conv4b'))
    model.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2), padding='valid', name='pool4'))
    
    model.add(Conv3D(512, 3, activation='relu', padding='same', name='conv5a'))
    model.add(Conv3D(512, 3, activation='relu', padding='same', name='conv5b'))
    model.add(ZeroPadding3D(padding=(0,1,1)))
    model.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2), padding='valid', name='pool5'))
    
    model.add(Flatten())
    
    model.add(Dense(4096, activation='relu', name='fc6'))
    model.add(Dropout(0.5))
    model.add(Dense(4096, activation='relu', name='fc7'))
    model.add(Dropout(0.5))
    model.add(Dense(487, activation='softmax', name='fc8'))

    if weights == 'sports1M':
        weights_path = get_file('sports1M_weights_tf.h5',
                                WEIGHTS_PATH,
                                cache_subdir='models')
        
        model.load_weights(weights_path)
    
    return model

In [16]:

#from c3d import C3D
#from sports1M_utils import preprocess_input, decode_predictions

model = C3D(weights='sports1M')


320020480/320010428 [==============================] - 5s 0us/step


In [17]:
#This function returns all the possible viewpoints of the given video along with the no. of occurrences
#Returns a dictionary 

def category1(vid):
  le=len(vid)
  le=le-16
  
  
  clip=vid[0:1]
  ot=[]
  thisdict=[""]
  thisdict.clear()
  
  y=0
  x = 0
  while (y <= le): 
    
    clip=vid[y: y +16]
    x = preprocess_input(clip)
    preds = model.predict(x)
    
    #If the confidence score is above 5% then only it will be considered as a viewpoint
    if decode_predictions(preds)[0][1]>0.05:
      thisdict.append(decode_predictions(preds)[0][0])
      

    y = y + 16
  return thisdict


In [18]:
# final_dic stores all the possible viewpoints wtih their frequencies
final_dic = {}

In [19]:
# adds local dict items to final_dict
from collections import Counter

def merge_dict(dict1 , dict2):
    dict3 = Counter(dict1) + Counter(dict2)
    return dict3   

In [20]:
listOfFiles

['/content/drive/MyDrive/vp_pro/view_point_dataset/Running/6eCgHq_7GPU.mp4',
 '/content/drive/MyDrive/vp_pro/view_point_dataset/Running/jnv0O9riZqc.mp4',
 '/content/drive/MyDrive/vp_pro/view_point_dataset/Running/rClB45bU4YE.mp4']

In [21]:
cnt=0
while(cnt<nov):


	path_dir=listOfFiles[cnt]
	vid = skvideo.io.vread(path_dir)
	arr=[""]
	arr=category1(vid)
	del vid


# mp is the local dict to store possible viewpoint and their frequency 
	mp = {}
	mp.clear()
	for i in range(len(arr)):
			if arr[i] in mp.keys():
				mp[arr[i]] += 1
			else:
				mp[arr[i]] = 1
	final_dic=merge_dict(final_dic,mp)			
				

	for x in mp:
		print(x, " ", mp[x])

	
  
	cnt=cnt+1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: DeprecationWarning:     `imresize` is deprecated!
    `imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
    Use ``skimage.transform.resize`` instead.


16384/5774 [=====================================================================================] - 0s 0us/step
radio-controlled car   11
isle of man tt   1
kabaddi   13
knife throwing   14
jujutsu   4
kyūdō   3
bocce   1
running   31
grappling   13
climbing   7
shooting   3
geocaching   1
hide-and-seek   8
boarding   17
bicycle   3
cycling   11
skating   2
racing   3
english pleasure   2
lacrosse   3
slacklining   1
race of champions   6
flyball   6
skimboarding   6
sailing   6
skipping rope   1
bike riding   3
football   1
folkrace   2
skeleton (sport)   2
tractor pulling   1
fishing   7
dragon boat   3
whitewater kayaking   1
angling   2
volleyball   3
noodling   1
bobsleigh   1
physical fitness   6
surfing   1
radio-controlled car   1
running   15
hunting   1
grappling   3
sailing   2
skiing   1
swimming   4
powerbocking   2
skating   2
iaidō   4
jujutsu   1
strength athletics   1
bicycle   1
english pleasure   1
model aircraft   2
racing   3
cycling   13
boarding   10
freestyle f

In [22]:
#final_dic

In [23]:
#sort the final_dic to get the viewpoint having most occurrences
final_dic = sorted(final_dic.items(), key=lambda x: x[1], reverse=True)

In [24]:
#final dictonary with all viewpoint in sorted order
final_dic

[('running', 115),
 ('boarding', 27),
 ('cycling', 24),
 ('radio-controlled car', 22),
 ('sailing', 22),
 ('grappling', 20),
 ('knife throwing', 17),
 ('freestyle football', 16),
 ('fishing', 14),
 ('kabaddi', 13),
 ('skating', 10),
 ('flyball', 10),
 ('hide-and-seek', 8),
 ('bicycle', 8),
 ('skimboarding', 8),
 ('climbing', 7),
 ('powerbocking', 7),
 ('racing', 6),
 ('race of champions', 6),
 ('physical fitness', 6),
 ('hapkido', 6),
 ('jujutsu', 5),
 ('model aircraft', 5),
 ('swimming', 4),
 ('iaidō', 4),
 ('kyūdō', 3),
 ('shooting', 3),
 ('english pleasure', 3),
 ('lacrosse', 3),
 ('bike riding', 3),
 ('dragon boat', 3),
 ('volleyball', 3),
 ('slacklining', 2),
 ('folkrace', 2),
 ('skeleton (sport)', 2),
 ('angling', 2),
 ('noodling', 2),
 ('footbag', 2),
 ('snorkeling', 2),
 ('roller derby', 2),
 ('isle of man tt', 1),
 ('bocce', 1),
 ('geocaching', 1),
 ('skipping rope', 1),
 ('football', 1),
 ('tractor pulling', 1),
 ('whitewater kayaking', 1),
 ('bobsleigh', 1),
 ('surfing', 1),

In [25]:
#returns the element(viewpoint) having most frequency
final_dic[0]

('running', 115)

In [26]:
#not needed for first run
!rm -rf '/content/drive/MyDrive/vp_pro/out'
!rm -rf '/content/drive/MyDrive/vp_pro/ext'
!rm -rf '/content/drive/MyDrive/vp_pro/input'

!mkdir '/content/drive/MyDrive/vp_pro/out'
!mkdir '/content/drive/MyDrive/vp_pro/ext'
!mkdir '/content/drive/MyDrive/vp_pro/input'

In [27]:
#del dir_list

In [28]:
# directory list
dir_list = []

In [29]:
# making folder for each video
root_path = '/content/drive/MyDrive/vp_pro/out'


s='vidd'
ss=''
cnt=0
while(cnt<nov):
	ss=s + str(cnt)
	dir_list.append(ss)
	cnt=cnt+1

print(dir_list)

for items in dir_list:
	path = os.path.join(root_path, items)
	os.mkdir(path)

['vidd0', 'vidd1', 'vidd2']


In [30]:
del dir_list
dir_list=[]

In [31]:
root_path = '/content/drive/MyDrive/vp_pro/ext'


s='vidd'
ss=''
cnt=0
while(cnt<nov):
	ss=s + str(cnt)
	dir_list.append(ss)
	cnt=cnt+1

print(dir_list)

for items in dir_list:
	path = os.path.join(root_path, items)
	os.mkdir(path)

['vidd0', 'vidd1', 'vidd2']


In [32]:
#del ot
#del fot
#ot array defines which frame to pick for the concluding video
# "0" defines frame not to consider and "1" defines frames to consider
ot=[]
# fot contains all ot array for the given set of videos
fot=[]
final_ot=[]

In [33]:
import cv2
import numpy as np
import os

from os.path import isfile, join

def convert_frames_to_video(pathIn,pathOut,fps):
    frame_array = []
    files = [f for f in os.listdir(pathIn) if isfile(join(pathIn, f))]

    #for sorting the file names properly
    #files.sort(key = lambda x: int(x[5:-4]))

    for i in range(len(files)):
        filename=pathIn + files[i]
        #reading each files
        img = cv2.imread(filename)
        height, width, layers = img.shape
        size = (width,height)
        print(filename)
        #inserting the frames into an image array
        frame_array.append(img)

    out = cv2.VideoWriter(pathOut,cv2.VideoWriter_fourcc(*'DIVX'), fps, size)

    for i in range(len(frame_array)):
        # writing to a image array
        out.write(frame_array[i])
    out.release()

In [34]:
cnt=0
while(cnt<nov):
  path_dir=listOfFiles[cnt]
  vid = skvideo.io.vread(path_dir)

  clip=vid[0:1]
  del ot
  ot=[]
  y=0

  while (y <= len(vid)-16):
    clip=vid[y: y +16]
    x = preprocess_input(clip)
    preds = model.predict(x)
    
    
    
    if decode_predictions(preds)[0][0]==final_dic[0][0]:
         
      for i in range(16):
        ot.append(1)
    else:
      
      for i in range(16):
        ot.append(0)

    y = y + 16

  tt=0
  while tt<16:
    ot.append(0)
    tt=tt+1

  del vid
  del clip
  fot.append(ot)
  
  



  
  vidcap = cv2.VideoCapture(listOfFiles[cnt])
  success,image = vidcap.read()
  count = 0

  

  
 



  while success:
    if ot[count]==1:
      cv2.imwrite("/content/drive/MyDrive/vp_pro/out/"+dir_list[cnt]+"/%d.jpg" % count, image)           
      success,image = vidcap.read()
      print('Read a new frame: ', success)
    else :
      cv2.imwrite("/content/drive/MyDrive/vp_pro/ext/"+dir_list[cnt]+"/%d.jpg" % count, image)     
      success,image = vidcap.read()
      print('Read a new name: ', success)

    count =count+ 1

    



  pathIn= "/content/drive/MyDrive/vp_pro/out/"+dir_list[cnt]+'/'
  pathOut = '/content/drive/MyDrive/vp_pro/vid_out%d.avi' %cnt
  fps = 30.0
  convert_frames_to_video(pathIn, pathOut, fps)

  
 










  




  
  cnt=cnt+1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: DeprecationWarning:     `imresize` is deprecated!
    `imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
    Use ``skimage.transform.resize`` instead.


Streaming output truncated to the last 5000 lines.
Read a new name:  True
Read a new name:  True
Read a new name:  True
Read a new name:  True
Read a new name:  True
Read a new name:  True
Read a new name:  True
Read a new name:  True
Read a new name:  True
Read a new name:  True
Read a new name:  True
Read a new name:  True
Read a new name:  True
Read a new name:  True
Read a new name:  True
Read a new name:  True
Read a new name:  True
Read a new name:  True
Read a new name:  True
Read a new name:  True
Read a new name:  True
Read a new name:  True
Read a new name:  True
Read a new name:  True
Read a new name:  True
Read a new name:  True
Read a new name:  True
Read a new name:  True
Read a new name:  True
Read a new name:  True
Read a new name:  True
Read a new name:  True
Read a new name:  True
Read a new name:  True
Read a new name:  True
Read a new name:  True
Read a new name:  True
Read a new name:  True
Read a new name:  True
Read a new name:  True
Read a new name:  True
Read a

In [35]:
print(fot)

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [36]:
"""
with open("/content/drive/MyDrive/out.txt", "w+") as f:
  data = f.read()
  f.write(str(ot))
"""

'\nwith open("/content/drive/MyDrive/out.txt", "w+") as f:\n  data = f.read()\n  f.write(str(ot))\n'